In [13]:
import lib.data.dataset as ds
import importlib
from torch.utils import data
importlib.reload(ds)
import torch.nn as nn
from easydict import EasyDict as edict
import torch

# 准备数据

In [47]:

# data_path = 'D:/PROJECT_TW/git/data/nlp/w2v/data/'
# model_path = 'D:/PROJECT_TW/git/data/nlp/w2v/'
model_path = '/home/hecong/temp/data/txtdect/'
data_path = '/home/hecong/temp/data/txtdect/data/'
eds = ds.EduData(data_path, model_path)
dl = data.DataLoader(eds,batch_size=50,shuffle=True)

房产 房产
股票 股票
教育 教育
彩票 彩票
财经 财经
家居 家居


In [48]:
#检测 nn.Emmeding 使用训练好的词向量结果
import torch.nn as nn
_,in_data = next(enumerate(dl))
words, labels = in_data
print(words.size())
embedding_matrix = torch.from_numpy(eds.embedding_matrix)
embed = nn.Embedding(40000, 20,_weight=embedding_matrix)
out_data = embed(words)

torch.Size([50, 64])


In [49]:
print(out_data[0][5])
print(words[0])
print(eds.idx_to_word[4237])
print(eds.word_vec_mod.wv.get_vector('布莱克本'))

tensor([-0.1263,  0.0895,  0.1022, -0.0152, -0.0142, -0.1604,  0.0136,  0.2782,
         0.1092,  0.3852,  0.0381, -0.1342, -0.0961, -0.3812,  0.1454, -0.0697,
         0.2485,  0.3764,  0.2450,  0.1813],
       dtype=torch.float64, grad_fn=<SelectBackward>)
tensor([ 7716,  8260,  9889,  3648,  6989, 10345,  2932,  2932,  2932,  2932,
         2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,
         2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,
         2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,
         2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,
         2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,  2932,
         2932,  2932,  2932,  2932])
桥往
[-0.09392507  0.07842945  0.08145787 -0.06175676 -0.01244122 -0.15300395
  0.00825114  0.26267716  0.12067892  0.3600079   0.05808657 -0.11769318
 -0.05155514 -0.31181234  0.12862869 -0.05610953  0.19461681  0.33628106
  0.21073024  

# 模型

In [50]:
# 初始化参数
opt =  edict()
opt.vocab_size = 40000
opt.embedding_dim = 20
opt.inception_dim = 40
opt.static = False
opt.num_classes = 6
opt.content_seq_len = 64
opt.linear_hidden_size = 256

In [73]:
# 关于conv1D 参看： https://blog.csdn.net/sunny_xsc1994/article/details/82969867
import lib.models.CNNText_inception as cni
import torch
import os
importlib.reload(cni)
model_path = '/home/hecong/temp/data/txtdect/text_label_model.pkl'
embedding_matrix = torch.from_numpy(eds.embedding_matrix)
model = cni.CNNText_inception(opt, embedding_matrix)
# 注意下面这个， 将model设置成model.double
# https://stackoverflow.com/questions/49407303/runtimeerror-expected-object-of-type-torch-doubletensor-but-found-type-torch-fl
model = model.double()
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))


# 训练

In [70]:
EPCHO = 100
loss_fun =  nn.CrossEntropyLoss()

LR = 0.001
# optimizer = torch.optim.Adam(model.parameters(), lr=LR)
MOMENTUM = 0.9
optimizer = torch.optim.SGD(
    model.parameters(), lr=LR, momentum=MOMENTUM)

for step in range(EPCHO):
    for i, (words, labels) in enumerate(dl):
        out = model(content=words)
        loss = loss_fun(out,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 20 == 0:
            print('{} - {} loss -->  {} '.format(step,i, loss))
            torch.save(model.state_dict(), model_path)
            
    
    

0 - 0 loss -->  0.16328241401315668 
0 - 20 loss -->  0.05633893977000484 
0 - 40 loss -->  0.26951541884198543 
1 - 0 loss -->  0.12244422631829163 
1 - 20 loss -->  0.06169934083270395 
1 - 40 loss -->  0.1322342433980609 
2 - 0 loss -->  0.11293894033177489 
2 - 20 loss -->  0.09090075843086266 
2 - 40 loss -->  0.15550820445095412 
3 - 0 loss -->  0.1764132696415878 
3 - 20 loss -->  0.12626639566555078 
3 - 40 loss -->  0.18832226897916585 
4 - 0 loss -->  0.032558019594395866 
4 - 20 loss -->  0.07329850720954112 
4 - 40 loss -->  0.121924726046223 
5 - 0 loss -->  0.07428776664816882 
5 - 20 loss -->  0.2830046885585898 
5 - 40 loss -->  0.1376136573303242 
6 - 0 loss -->  0.15969451648023142 
6 - 20 loss -->  0.12167635734823863 
6 - 40 loss -->  0.10387805624821288 
7 - 0 loss -->  0.15330603433696463 
7 - 20 loss -->  0.12172527835127418 
7 - 40 loss -->  0.11771297107651428 
8 - 0 loss -->  0.121118666147443 
8 - 20 loss -->  0.07305324908084934 
8 - 40 loss -->  0.139536910

KeyboardInterrupt: 

# 验证

In [87]:
import jieba
testDoc = '''
    中国2010年棉花产量预计减少5.5%
　　新浪财经讯 11月18日下午消息，来自国资委网站消息，中国储备棉管理总公司周四发布报告称，由于遭遇恶劣天气，预计今年全国棉花种植总产量636.0万吨，减少5.5%。
　　2010年10月下旬至11月上旬，中国储备棉管理总公司承建的国家棉花市场监测系统对2010年度我国棉花产量情况在全国范围内展开调查，涉及15个植棉省(自治区)、87个植棉县(市、团场)。
　　结果显示，由于今年棉花播种期遭受低温、吐絮期遭遇连续强降雨等灾害天气，棉花质量和产量均受到不利影响。今年全国棉花种植面积7568.2万亩，较上年减少1.1%；单产84.0公斤/亩，下降4.4%；总产量636.0万吨，减少5.5%。
　　该项统计数为初步预计结果，报告呼吁棉花产业各环节对棉花产量和供求格局有清醒认识，克服投机心理，谨慎经营，控制风险。 
'''    
# 提取中文, 分词， 得到词向量
line = ds.extract_chinese(testDoc)
ws = jieba.lcut(line, cut_all=False, HMM=True)
words_ids = [eds.word_to_idx[x] for x in ws if x in eds.word_to_idx]
words_ids = words_ids[0:64]
words_ids = torch.LongTensor(words_ids)
words_ids = words_ids.unsqueeze(0)
twords = words[0]
twords = twords.unsqueeze(0)
model.eval()
out = model(content=words_ids)
print('类别 --> {}'.format(eds.index_lables[torch.argmax(out).item()]))

类别 --> 财经
